Unique values

In [6]:
import pandas as pd

# Load the dataset
file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\Chennai Sales Data.csv"
data = pd.read_csv(file_path)

# Identify categorical columns and exclude date columns
# Assuming that date columns are those which have 'date' in their name
categorical_columns = [col for col in data.columns if data[col].dtype == 'object' and 'date' not in col.lower()]

# Create a dictionary to store unique values for each categorical column
unique_values_dict = {}

for col in categorical_columns:
    unique_values = data[col].unique()
    unique_values_dict[col] = unique_values

# Create a DataFrame to store unique values
# Use a DataFrame with each column containing unique values of corresponding categorical column
unique_values_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in unique_values_dict.items()]))

# Save the DataFrame to a CSV file
output_file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\categorical_unique_values.csv"
unique_values_df.to_csv(output_file_path, index=False)

print("Unique values for categorical columns have been saved to", output_file_path)



Unique values for categorical columns have been saved to D:\Guvi\Own_Projects\House Price Prediction\categorical_unique_values.csv


Correcting the Spelling Mistakes 

In [9]:
import pandas as pd

# Define corrections for categorical columns
corrections = {
    'AREA': {
        'Karapakam': 'Karapakkam',
        'Chrompt': 'Chrompet',
        'Chrmpet': 'Chrompet',
        'Ana Nagar': 'Anna Nagar',
        'Adyr': 'Adyar',
        'Velchery': 'Velachery',
        'Ann Nagar': 'Anna Nagar',
        'KKNagar': 'KK Nagar',
        'Karapakam': 'Karapakkam',
        'Chormpet': 'Chrompet'
    },
    'SALE_COND': {
        'AbNormal': 'Ab Normal',
        'Ab Normal': 'Ab Normal',
        'Partiall': 'Partial',
        'Adj Land': 'AdjLand',
        'PartiaLl': 'Partial'
    },
    'PARK_FACIL': {
        'Noo': 'No'
    },
    'BUILDTYPE': {
        'Comercial': 'Commercial',
        'Other': 'Others'
    },
    'UTILITY_AVAIL': {
        'ELO': 'AllPub',
        'NoSewr': 'NoSeWa',
        'NoSeWa': 'NoSeWa',
        'All Pub': 'AllPub'
    },
    'STREET': {
        'Pavd': 'Paved',
        'NoAccess': 'No Access'
    }
}

# Load the dataset
file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\Chennai Sales Data.csv"
house_data = pd.read_csv(file_path)

# Apply corrections to categorical columns
for col, mapping in corrections.items():
    if col in house_data.columns:
        house_data[col] = house_data[col].replace(mapping)

# Save the corrected data to a new CSV file
corrected_file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\Chennai_Sales_Data_Corrected.csv"
house_data.to_csv(corrected_file_path, index=False)

print("Corrected file is saved.")


Corrected file is saved.


Data Preprocessing

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Load the corrected dataset
file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\Chennai_Sales_Data_Corrected.csv"
house_data = pd.read_csv(file_path)

# Convert date columns to datetime format
house_data['DATE_SALE'] = pd.to_datetime(house_data['DATE_SALE'], format='%d-%m-%Y')
house_data['DATE_BUILD'] = pd.to_datetime(house_data['DATE_BUILD'], format='%d-%m-%Y')

# Extract useful components from date columns
house_data['SALE_YEAR'] = house_data['DATE_SALE'].dt.year
house_data['SALE_MONTH'] = house_data['DATE_SALE'].dt.month
house_data['BUILD_YEAR'] = house_data['DATE_BUILD'].dt.year
house_data['BUILD_AGE'] = house_data['SALE_YEAR'] - house_data['BUILD_YEAR']

# Drop original date columns if they are no longer needed
house_data = house_data.drop(columns=['DATE_SALE', 'DATE_BUILD'])

# Drop features that are less likely to help in prediction or may lead to overfitting
# For instance, identifiers and features with too many categories or minimal variance
house_data = house_data.drop(columns=['PRT_ID', 'SALE_COND', 'STREET', 'MZZONE', 'UTILITY_AVAIL'])

# Handle Missing Values
num_cols = house_data.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='mean')
house_data[num_cols] = imputer.fit_transform(house_data[num_cols])

# Encode Categorical Variables
cat_cols = house_data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    house_data[col] = le.fit_transform(house_data[col])
    label_encoders[col] = le

# Save the label encoders as a joblib file
label_encoder_file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\label_encoders.pkl"
joblib.dump(label_encoders, label_encoder_file_path)

# Save preprocessed data
output_file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\preprocessed_house_data.csv"
house_data.to_csv(output_file_path, index=False)

print("Preprocessed file and label encoders are saved.")



Preprocessed file and label encoders are saved.


Feature Selection

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load preprocessed data
house_data = pd.read_csv(r"D:\Guvi\Own_Projects\House_Price_Prediction\preprocessed_house_data.csv")

# Define features and target variable
X = house_data.drop(columns=['SALES_PRICE'])
y = house_data['SALES_PRICE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model to assess feature importance (RandomForestRegressor used here as an example)
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Feature importance analysis
importances = model.feature_importances_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
print("Feature Importance:\n", feature_importance.sort_values(by='Importance', ascending=False))

# Optionally: Drop features with very low importance if desired
threshold = 0.01  # Example threshold for feature importance
important_features = feature_importance[feature_importance['Importance'] > threshold]['Feature']
X = X[important_features.tolist()]

print("Feature selection complete. Selected features saved.")


Feature Importance:
           Feature  Importance
12        REG_FEE    0.770348
0            AREA    0.075496
1        INT_SQFT    0.055797
7       BUILDTYPE    0.035141
3       N_BEDROOM    0.015695
4      N_BATHROOM    0.010283
13         COMMIS    0.009797
2   DIST_MAINROAD    0.003198
11     QS_OVERALL    0.002937
8        QS_ROOMS    0.002755
9     QS_BATHROOM    0.002733
6      PARK_FACIL    0.002572
10     QS_BEDROOM    0.002524
14      SALE_YEAR    0.002384
16     BUILD_YEAR    0.002287
17      BUILD_AGE    0.002267
15     SALE_MONTH    0.002081
5          N_ROOM    0.001705
Feature selection complete. Selected features saved.


Model Training For House Price Prediction

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import time

# Load the preprocessed data
file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\preprocessed_house_data.csv"
house_data = pd.read_csv(file_path)

# Define features and target variable based on selected features
selected_features = [
    'REG_FEE', 'AREA', 'INT_SQFT', 'BUILDTYPE', 'N_BEDROOM', 'N_BATHROOM',
    'COMMIS', 'DIST_MAINROAD', 'QS_OVERALL', 'QS_ROOMS', 'QS_BATHROOM',
    'PARK_FACIL', 'QS_BEDROOM', 'SALE_YEAR', 'BUILD_YEAR', 'BUILD_AGE',
    'SALE_MONTH', 'N_ROOM'
]
X = house_data[selected_features]
y = house_data['SALES_PRICE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models and hyperparameters for Grid Search
models = {
    'RandomForest': RandomForestRegressor(),
    'Lasso': Lasso(),
    'Ridge': Ridge()
}

param_grids = {
    'RandomForest': {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    'Lasso': {
        'alpha': [0.01, 0.1, 1, 10]
    },
    'Ridge': {
        'alpha': [0.01, 0.1, 1, 10]
    }
}

# Perform Grid Search and Train Models
best_models = {}
training_times = {}
prediction_times = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    start_train_time = time.time()
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    end_train_time = time.time()
    
    training_time = end_train_time - start_train_time
    training_times[model_name] = training_time
    
    # Evaluate the model
    start_pred_time = time.time()
    y_pred = best_models[model_name].predict(X_test)
    end_pred_time = time.time()
    
    prediction_time = end_pred_time - start_pred_time
    prediction_times[model_name] = prediction_time
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{model_name} MSE: {mse}")
    print(f"{model_name} MAE: {mae}")
    print(f"{model_name} R²: {r2}")
    print(f"{model_name} Training Time: {training_time:.4f} seconds")
    print(f"{model_name} Prediction Time: {prediction_time:.4f} seconds")

# Save the best model (RandomForest in this case, assuming it's the best)
best_model = best_models['RandomForest']
model_file_path = r"D:\Guvi\Own_Projects\House_Price_Prediction\best_model.pkl"
joblib.dump(best_model, model_file_path)

print("Best model is saved.")


Training RandomForest...
RandomForest MSE: 561605079450.1067
RandomForest MAE: 597025.0541815774
RandomForest R²: 0.9572514389751576
RandomForest Training Time: 598.0848 seconds
RandomForest Prediction Time: 0.0625 seconds
Training Lasso...
Lasso MSE: 1901049798438.0662
Lasso MAE: 1103898.1108480382
Lasso R²: 0.8552948570205839
Lasso Training Time: 1.1375 seconds
Lasso Prediction Time: 0.0000 seconds
Training Ridge...


c:\Users\sripathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.73456e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\sripathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.6071e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\sripathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.68839e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
c:\Users\sripathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.66573e-17): result may not be accurate.
  return l

Ridge MSE: 1901082187476.7996
Ridge MAE: 1103906.090797813
Ridge R²: 0.8552923916141097
Ridge Training Time: 0.3884 seconds
Ridge Prediction Time: 0.0000 seconds
Best model is saved.
